In [32]:
from rdflib import Graph, Namespace
from pyvis.network import Network
from IPython.display import IFrame, display

# 1. Adjust the path to your TTL file
TTL_FILE = "../linkml/data/rdf/epd_rdf_instance_datastore_canonical_skos_din_bki_shacl.ttl"

# 2. Create and parse the graph
g = Graph()
g.parse(TTL_FILE, format="turtle")

# 3. Define namespaces
ILCD = Namespace("https://example.org/ilcd/")
DIN  = Namespace("https://example.org/din276/")
CC   = Namespace("https://example.org/concreteclass/")
OBD  = Namespace("https://example.org/obd/")
BKI  = Namespace("https://example.org/bki/")

# 4. Find a subset of EPDs
EPD_LIMIT = 2
q_epds = f"""
SELECT ?epd
WHERE {{
  ?epd a <{ILCD}ProcessDataSet> .
}}
LIMIT {EPD_LIMIT}
"""
results_epd = g.query(q_epds)
epd_uris = [str(row.epd) for row in results_epd]

# ---------------------------------------------------------------------------------------
# Create the PyVis network
# ---------------------------------------------------------------------------------------
net = Network(
    notebook=True,
    height="1080px",
    width="1920px",
    directed=False,
    cdn_resources="in_line",
)

# Hierarchical layout
# net.set_options(
#     """
# var options = {
#   "layout": {
#     "hierarchical": {
#       "enabled": true,
#       "levelSeparation": 200,
#       "nodeSpacing": 150,
#       "treeSpacing": 50,
#       "direction": "UD",
#       "sortMethod": "directed"
#     }
#   },
#   "physics": {
#     "enabled": false
#   },
#   "edges": {
#     "smooth": {
#       "enabled": true,
#       "type": "continuous", 
#       "forceDirection": "none", 
#       "roundness": 0.3
#     },
#     "font": {
#       "vadjust": -5
#     }
#   }
# }
# """
# )


# Force Atlas 2
net.force_atlas_2based(
    central_gravity=0.1,
    spring_length=2000,
    overlap=1
)

# net.barnes_hut(
#     central_gravity=0.3,
#     spring_length=250,
#     overlap=1
# )

# Disable physics
net.toggle_physics(False)

# Show interactive physics controls
# net.show_buttons(filter_=['physics'])


epd_number = 0

for epd_uri in epd_uris:
    epd_number += 1
    
    # ------------------------------------------------------------------
    # LEVEL 0: EPD node
    # ------------------------------------------------------------------
    net.add_node(
        epd_uri, 
        label=f"EPD {epd_number}", 
        shape="ellipse", 
        color="#a2d2ff",
        level=0
    )

    # ------------------------------------------------------------------
    # 1) DIN 276 cost groups: only 322, 331
    # ------------------------------------------------------------------
    q_din = f"""
    SELECT ?costgroup
    WHERE {{
      <{epd_uri}> <{DIN}hasDIN276CostGroup> ?costgroup .
    }}
    """
    results_din = g.query(q_din)
    for row in results_din:
        cg_uri = str(row.costgroup)
        cg_label = cg_uri.split("/")[-1]
        # Skip if not costgroup_322 or costgroup_331
        if cg_label not in ["costgroup_322", "costgroup_331"]:
            continue

        # We'll place cost groups at level=1
        net.add_node(
            cg_uri, 
            label=cg_label, 
            shape="box", 
            color="#fef9c3", 
            level=1
        )
        net.add_edge(epd_uri, cg_uri, label="hasDIN276CostGroup")

        # If costgroup_331, include BKI elements referencing it
        if cg_label == "costgroup_322":
            q_bki = f"""
            SELECT ?bkiElem ?bkiName
            WHERE {{
              bki:element_291c4af9512d41538ca62691929c1d0f a <{BKI}BKIElement> ;
                       <{BKI}name> ?bkiName ;
                       <{DIN}hasDIN276CostGroup> <{cg_uri}> .
            }}
            """
            bki_results = g.query(q_bki)
            for bki_row in bki_results:
                bki_elem  = str(bki_row.bkiElem)
                bki_label = str(bki_row.bkiName)
                
                # Put BKI elements at level=2
                net.add_node(
                    bki_elem, 
                    label=bki_label, 
                    shape="box", 
                    color="#d9f7be",
                    level=4
                )
                net.add_edge(cg_uri, bki_elem, label="hasBKIElement")

    # ------------------------------------------------------------------
    # 2) Strength / Weight Classification
    # ------------------------------------------------------------------
    q_strength = f"""
    SELECT ?strength
    WHERE {{
      <{epd_uri}> <{CC}hasStrengthClassification> ?strength .
    }}
    """
    strength_results = g.query(q_strength)
    for srow in strength_results:
        strength_uri = str(srow.strength)
        s_label = strength_uri.split("/")[-1]
        net.add_node(
            strength_uri, 
            label=s_label, 
            shape="box", 
            color="#fde2e4",
            level=3
        )
        net.add_edge(epd_uri, strength_uri, label="hasStrengthClassification")

    q_weight = f"""
    SELECT ?weight
    WHERE {{
      <{epd_uri}> <{CC}hasWeightClassification> ?weight .
    }}
    """
    weight_results = g.query(q_weight)
    for wrow in weight_results:
        weight_uri = str(wrow.weight)
        w_label = weight_uri.split("/")[-1]
        net.add_node(
            weight_uri, 
            label=w_label, 
            shape="box", 
            color="#fde2e4",
            level=3
        )
        net.add_edge(epd_uri, weight_uri, label="hasWeightClassification")

    # ------------------------------------------------------------------
    # 3) classificationInformation => level=1
    # ------------------------------------------------------------------
    q_classInfo = f"""
    SELECT ?classInfo
    WHERE {{
      <{epd_uri}> <{ILCD}processInformation> ?procInfo .
      ?procInfo <{ILCD}dataSetInformation> ?dataSetInfo .
      ?dataSetInfo <{ILCD}classificationInformation> ?classInfo .
    }}
    """
    cinfo_results = g.query(q_classInfo)

    for ci_row in cinfo_results:
        ci_uri = str(ci_row.classInfo)
        net.add_node(
            ci_uri, 
            label="classificationInformation", 
            shape="ellipse", 
            color="#e7e7e7",
            level=1
        )
        net.add_edge(epd_uri, ci_uri, label="hasClassificationInfo")

        # Classification nodes => level=2
        q_classifications = f"""
        SELECT ?classification ?className
        WHERE {{
          <{ci_uri}> <{ILCD}classification> ?classification .
          OPTIONAL {{ ?classification <{ILCD}name> ?className . }}
        }}
        """
        sub_results = g.query(q_classifications)
        for sub_row in sub_results:
            cls_uri = str(sub_row.classification)
            cls_name = sub_row.className if sub_row.className else "Classification"

            net.add_node(
                cls_uri, 
                label=cls_name, 
                shape="box", 
                color="#ccccee",
                level=2
            )
            net.add_edge(ci_uri, cls_uri, label="classification")

            # Decide how to handle entries
            skip_entries = False
            keep_only_rc_beton = False
            lower_cls = cls_name.lower()
            if "epdnorge" in lower_cls or "ibucategories" in lower_cls:
                skip_entries = True
            elif "oekobau" in lower_cls:
                keep_only_rc_beton = True

            if skip_entries:
                continue

            # classification entries => level=3
            q_entries = f"""
            SELECT ?entry ?classVal ?canonCat
            WHERE {{
              <{cls_uri}> <{ILCD}classEntries> ?entry .
              ?entry <{ILCD}value> ?classVal .
              OPTIONAL {{ ?entry <{OBD}hasCanonicalCategory> ?canonCat . }}
            }}
            """
            entry_results = g.query(q_entries)
            for e_row in entry_results:
                entry_uri = str(e_row.entry)
                entry_val = str(e_row.classVal)

                if keep_only_rc_beton:
                    # Only keep "Ready mixed concrete" or "Beton"
                    if entry_val not in ["Ready mixed concrete", "Beton"]:
                        continue

                net.add_node(
                    entry_uri, 
                    label=entry_val, 
                    shape="ellipse", 
                    color="#ffffcc",
                    level=3
                )
                net.add_edge(cls_uri, entry_uri, label="classEntries")

                # canonical category => level=4
                if e_row.canonCat:
                    canon_uri = str(e_row.canonCat)
                    # fetch SKOS label
                    q_canonLabel = f"""
                    SELECT ?pref
                    WHERE {{
                      <{canon_uri}> <http://www.w3.org/2004/02/skos/core#prefLabel> ?pref .
                      FILTER(lang(?pref) = "en")
                    }}
                    LIMIT 1
                    """
                    canon_label_res = g.query(q_canonLabel)
                    canon_label = None
                    for clrow in canon_label_res:
                        canon_label = str(clrow.pref)
                    if not canon_label:
                        canon_label = canon_uri.split("/")[-1]

                    net.add_node(
                        canon_uri, 
                        label=canon_label, 
                        shape="ellipse", 
                        color="#ffeedb",
                        level=4
                    )
                    net.add_edge(entry_uri, canon_uri, label="hasCanonicalCategory")

# ---------------------------------------------------------------------------------------
# Render & display
# ---------------------------------------------------------------------------------------
html_file = "knowledge_graph.html"
net.show(html_file)
display(IFrame(html_file, width="100%", height="100%"))


knowledge_graph.html
